In [31]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/laptop_data(github).csv")
df.head()


,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [32]:
df.drop(columns=['Unnamed: 0','ScreenResolution','Memory','TypeName'], inplace=True)


In [33]:
df['Ram'] = df['Ram'].str.replace('GB','').astype(int)
df['Weight'] = df['Weight'].str.replace('kg','').astype(float)
df['Cpu'] = df['Cpu'].apply(lambda x: x.split()[0])
df['Gpu'] = df['Gpu'].apply(lambda x: x.split()[0])


In [34]:
def os_cat(x):
    if 'Windows' in x:
        return 'Windows'
    elif 'Mac' in x:
        return 'Mac'
    else:
        return 'Other'

df['OpSys'] = df['OpSys'].apply(os_cat)


In [35]:
from sklearn.preprocessing import LabelEncoder

encoders = {}
cat_cols = ['Company','Cpu','Gpu','OpSys']

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X = df[['Company','Cpu','Gpu','OpSys','Ram','Weight','Inches']]
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)

model.fit(X_train, y_train)


RandomForestRegressor(n_estimators=300, random_state=42)

In [37]:
import joblib

joblib.dump(model, "laptop_price_model.pkl")
joblib.dump(encoders, "encoders.pkl")


['encoders.pkl']

In [38]:
import ipywidgets as widgets
from IPython.display import display
company = widgets.Dropdown(
    options=encoders['Company'].classes_,
    description='Company'
)

cpu = widgets.Dropdown(
    options=encoders['Cpu'].classes_,
    description='CPU'
)

gpu = widgets.Dropdown(
    options=encoders['Gpu'].classes_,
    description='GPU'
)

os = widgets.Dropdown(
    options=encoders['OpSys'].classes_,
    description='OS'
)

ram = widgets.IntText(description='RAM (GB)')
weight = widgets.FloatText(description='Weight (kg)')
inches = widgets.FloatText(description='Inches')

button = widgets.Button(description="Predict Price")
output = widgets.Output()


In [39]:
def predict_price(b):
    comp = encoders['Company'].transform([company.value])[0]
    cpu_val = encoders['Cpu'].transform([cpu.value])[0]
    gpu_val = encoders['Gpu'].transform([gpu.value])[0]
    os_val = encoders['OpSys'].transform([os.value])[0]

    data = np.array([[comp, cpu_val, gpu_val, os_val,
                      ram.value, weight.value, inches.value]])

    price = model.predict(data)[0]

    with output:
        output.clear_output()
        print(f"💻 Predicted Laptop Price: ₹ {round(price,2)}")

button.on_click(predict_price)


In [40]:
display(company, cpu, gpu, os, ram, weight, inches, button, output)


Dropdown(description='Company', options=('Acer', 'Apple', 'Asus', 'Chuwi', 'Dell', 'Fujitsu', 'Google', 'HP', …

Dropdown(description='CPU', options=('AMD', 'Intel', 'Samsung'), value='AMD')

Dropdown(description='GPU', options=('AMD', 'ARM', 'Intel', 'Nvidia'), value='AMD')

Dropdown(description='OS', options=('Mac', 'Other', 'Windows'), value='Mac')

IntText(value=0, description='RAM (GB)')

FloatText(value=0.0, description='Weight (kg)')

FloatText(value=0.0, description='Inches')

Button(description='Predict Price', style=ButtonStyle())

Output()

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [42]:
from google.colab import files
files.download('encoders.pkl')
files.download('laptop_price_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>